<a href="https://colab.research.google.com/github/okyandex/mit-dl-lab2-ru/blob/main/MIT_Music_Generation_RU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab1/solutions/Part2_Music_Generation_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab1/solutions/Part2_Music_Generation_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [ ]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Лабораторная работа 1: Введение в TensorFlow и создание музыки с помощью рекуррентныx нейронных сетей (RNN)

# Часть 2: Cоздание музыки с помощью RNN

В этой части мы рассмотрим создание рекуррентной нейронной сети для генерации музыки. Мы научим модель находить шаблоны в нотах, представленных в [ABC-нотации](https://ru.wikipedia.org/wiki/ABC_(%D1%8F%D0%B7%D1%8B%D0%BA_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%82%D0%BA%D0%B8)) и затем использовать её для создания музыки. 

## 2.1 Зависимости
Сначала загрузим репозиторий курса, установим зависимости и импортируем соответствующие пакеты, которые нам понадобятся для этой лабораторной работы.

In [ ]:
# Импорт Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

# Загрузка и импорт MIT 6.S191 пакета
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Импорт остальных пакетов
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1

# Проверьте, используете ли вы аппаратный ускоритель GPU.
#   Среда выполнения > Сменить среду выполнения > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

## 2.2 Набор данных

![Let's Dance!](http://33.media.tumblr.com/3d223954ad0a77f4e98a7b87136aa395/tumblr_nlct5lFVbF1qhu7oio1_500.gif)

Мы собрали набор данных из тысяч ирландских народных песен, представленных в нотации ABC. Давайте загрузим набор данных и посмотрим на него:


In [ ]:
# Загрузка набора данных
songs = mdl.lab1.load_training_data()

# Распечатаем одну из песен, чтобы изучить ее более детально
example_song = songs[0]
print("\nПример песни: ")
print(example_song)

Мы можем легко преобразовать песню из формата ABC в звуковую волну и воспроизвести ее. Подождите, пока это преобразование запустится, это может занять некоторое время.

In [ ]:
# Конвертируем песню из нотации ABC в аудиофайл и прослушаем его
mdl.lab1.play_song(example_song)

Одна важная вещь заключается в том, что эта нотация не просто содержит информацию о воспроизводимых нотах, но также содержит метаинформацию, такую как название песни, тональность и темп. Как количество различных символов, присутствующих в текстовом файле, влияет на сложность задачи обучения? Это станет важным вскоре, когда мы создадим числовое представление для текстовых данных.

In [ ]:
# Соединим наш список песен в одну строку, содержащую все песни
songs_joined = "\n\n".join(songs) 

# Найдём все уникальные символы в объединённой строке
vocab = sorted(set(songs_joined))
print("Уникальных символов в наборе данных: ", len(vocab))

## 2.3 Обработка набора данных для задачи обучения.

Рассмотрим нашу задачу прогнозирования. Мы пытаемся обучить модель находить шаблоны в музыке, представленной в нотации ABC, а затем использовать эту модель для создания (т.е. прогнозирования) нового музыкального произведения на основе этой информации.

На самом деле мы спрашиваем модель: учитывая символ или последовательность символов, какой наиболее вероятный следующий символ? Мы обучим модель выполнять эту задачу.

Для этого мы передадим в модель последовательность символов и обучим модель предсказывать выходные данные, то есть следующий символ на каждом временном отрезке. RNN поддерживает внутреннее состояние, которое зависит от увиденных ранее символов, поэтому информация обо всех символах, увиденных до данного момента, будет учтена при создании прогноза.

### Векторизация текста
До того как мы начнём обучать нашу RNN модель, нам нужно создать числовое представление нашего набора данных, основанного на буквах. Для этого мы создадим два словаря: один для перевода символов в числа, и второй для перевода чисел назад в символы. Напомним, что мы только что определили набор уникальных символов

In [ ]:
### Определим числовое представление текста ###

# Сделаем сопоставление символов и уникального индекса.
# Например, чтобы получить индекс симова "d", 
#   мы можем выполнить `char2idx["d"]`.  
char2idx = {u:i for i, u in enumerate(vocab)}

# Сделаем сопоставление от индексов к символам. Это работает наоборот
# в отличии от char2idx и позволяет нам
# по уникальному индексу получить символ в нашем словаре.
idx2char = np.array(vocab)

Это даёт нам числовое представление каждого символа. Обратите внимание, что уникальные символы (то есть наш словарь) в тексте отображаются как индексы от 0 до `len(unique)`. Давайте взглянем на это числовое представление нашего набора данных:

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
### Векторизация строки с песнями ###

''' TODO: Напишите функцию для преобразования строки, состоящей их всех песен в  
    числовое представление. Исользуйте соответствующий словарь,
    чтобы конвертировать текстовые символы в индексы.

    NOTE: Функия `vectorize_string` должна возвращать 
    np.array с `N` элементами, где `N` это количество символов 
    во входной строке
'''
def vectorize_string(string):
  vectorized_output = np.array([char2idx[char] for char in string])
  return vectorized_output

vectorized_songs = vectorize_string(songs_joined)

Мы также можем посмотреть, как часть текста преображается в численное представление:

In [ ]:
print ('{} ---- Символы преобразованные в числа ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# Проверьте что vectorized_songs является массивом numpy
assert isinstance(vectorized_songs, np.ndarray), "возвращаемый результат должен быть массивом numpy"

### Создание обучающих примеров и целей

Наш следующий шаг - фактически разделить текст на примеры последовательностей, которые мы будем использовать во время обучения. Каждая последовательность, которую мы передадим в нашу сеть будет содержать кол-во символов в тексте `seq_length`. Нам также нужно будет определить целевую последовательность для каждой входной последовательности, которая будет использоваться при обучении сети для предсказания следующего символа. Для каждого входа соответствующая цель будет содержать текст одинаковой длины, за исключением смещения на один символ вправо.

Для этого мы разобьем текст на куски (пакеты) длины `seq_length+1`. Предположим `seq_length` равна 4 и наш текст это "Hello". Тогда наша входная последовательность это "Hell" а целевая последовательность это "ello".

Затем такой пакетный метод позволит нам преобразовать этот поток индексов символов в последовательности желаемого размера.

In [ ]:
### Определение пакетов для создания обучающих примеров ###

def get_batch(vectorized_songs, seq_length, batch_size):
  # длина векторизированной строки с песнями
  n = vectorized_songs.shape[0] - 1
  # случайным образом выбираем начальные индексы для примеров в обучающем пакете
  idx = np.random.choice(n-seq_length, batch_size)

  '''TODO: Создать список входных последовательностей для обучающего пакета'''
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]

  '''TODO: Создать список выходных последовательностей для обучающего пакета'''
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]

  # x_batch, y_batch предоставляют истинные данные и цели для обучения сети
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch


# Выполним несколько простых тестов, чтобы убедиться, что функция
# работает правильно 
test_args = (vectorized_songs, 10, 2)
if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args): 
   print("======\n[ОШИБКА] тесты не пройдены")
else: 
   print("======\n[УСПЕХ] все тесты пройдены!")

Для каждого из этих векторов каждый индекс обрабатывается за один временной отрезок. Так, для входа на временном отрезке 0, модель получает индекс для первого символа в последовательности, и пытается предсказать индекс следующего символа. На следующем временном отрезке происходит то же самое, но RNN учитывает информацию с предыдущего шага, т.е. его обновленное состояние в дополнение к текущему вводу.

Мы можем рассмотреть это более подробно, взглянув как это работает с первыми несколькими символами в нашем тексте:

In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Шаг {:3d}".format(i))
    print("  вход: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  ожидаемый выход: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

## 2.4 Модель рекуррентной нейронной сети (RNN)

Теперь мы готовы создать и обучить модель RNN на нашем наборе музыкальных данных (ABC-нотация), и затем использовать эту обученную модель для создания новой музыки. Мы будем обучать нашу RNN, используя пакеты фрагментов песен из нашего набора данных, которые мы создали в предыдущем разделе.

Модель основана на архитектуре LSTM, где мы используем вектор состояния для сохранения информации о временных отношениях между последовательными символами. Окончательный выход LSTM затем подается в полносвязный [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) слой, выход которого будет обрабатываться с помощью функции softmax для каждого символа в словаре, а затем сделаем выборку из этого распределения, чтобы предсказать следующий символ.

Как мы уже рассказали в первой части этой лабораторной работы, мы будем использовать Keras API, в частности, [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential), чтобы создать модель. Для определения модели используются три слоя:

* [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding): Это входной слой, состоящий из таблицы, которая отображает номера каждого символа в вектор с размерностью `embedding_dim`.
* [`tf.keras.layers.LSTM`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM): Наша LSTM сеть, с размером `units=rnn_units`. 
* [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): Выходной слой, с выходным размером `vocab_size`.


<img src="https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab1/img/lstm_unrolled-01-01.png" alt="Drawing"/>

### Создадим нашу RNN модель

Теперь мы определим функцию, которую мы будем использовать для построения модели.

In [ ]:
def LSTM(rnn_units): 
  return tf.keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

Время пришло! Заполните TODO, чтобы создать модель RNN в функции build_model, а затем вызовите функцию, которую вы только что определили, чтобы создать экземпляр модели!

In [ ]:
### Создание RNN модели ###

'''TODO: Добавьте LSTM и Dense слои, чтобы создать RNN модель, 
    используя Sequential API.'''
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Слой 1: Embedding слой для преобразования индексов в dense векторы 
    #   фиксированного размера
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Слой 2: LSTM с `rnn_units` количество единиц.
    # TODO: Вызовите функцию LSTM, определенную выше, чтобы добавить этот слой.
    LSTM(rnn_units), 

    # Слой 3: Dense (полносвязный) слой, который преобразует выход LSTM
    #  в размер словаря. 
    # TODO: Добавьте Dense слой.
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

# Создайте простую модель с гиперпараметрами по умолчанию. Вы можете изменить
#   их позже.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

### Проверьте модель RNN

Всегда полезно выполнить несколько простых проверок нашей модели, чтобы убедиться, что она ведет себя так, как ожидалось.

Во-первых, мы можем использовать функцию `Model.summary` чтобы вывести краткое содержание внутренней работы нашей модели. Здесь мы можем проверить слои в модели, форму выхода каждого из слоев, размер партии и т.д.

In [ ]:
model.summary()

Мы также можем быстро проверить размерность нашего выхода, используя длину последовательности 100. Обратите внимание, что модель может быть запущена на входах любой длины.

In [ ]:
x, y = get_batch(vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
print("Входная форма:      ", x.shape, " # (batch_size, sequence_length)")
print("Форма предсказания: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

### Прогнозы от нетренированной модели

Давайте посмотрим, что предсказывает наша нетренированная модель.
Чтобы получить реальные прогнозы от модели, мы делаем выборку из выходного распределения, которое определяется функцией softmax по нашему словарю символов.
Это даст нам фактические индексы символов. Это означает, что мы используем [категориальное распределение](https://en.wikipedia.org/wiki/Categorical_distribution) для выборки на примерах предсказания. Это дает предсказание следующего символа (в частности, его индекса) на каждом временном отрезке.

Обратите внимание на то, что мы делаем выборку из этого распределения вероятностей, а не просто берём argmax, что может привести к застреванию модели в цикле.

Давайте попробуем сделать выборку для первого примера в пакете.

In [ ]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

Теперь мы можем их декодировать, чтобы увидеть текст, предсказанный нетренированной моделью:

In [ ]:
print("Input: \n", repr("".join(idx2char[x[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Как вы можете заметить, текст, сгенерированный нетренированной моделью выглядит довольно бессмысленно. Как мы можем это улучшить? Мы можем обучить сеть!

## 2.5 Обучение модели: функция потерь и обучающие действия

Пришло время обучить модель!

На данный момент мы можем рассматривать задачу предсказывания следующего символа как обычную задачу классификации. Учитывая предыдущее состояние RNN, а также вход на данном временном отрезке, мы хотим предсказать класс следующего символа -- то есть фактически предсказать следующий символ. 
Чтобы обучить нашу модель для решения этой задачи классификации, мы можем использовать форму функции потерь `crossentropy` (negative log likelihood loss).
В частности мы будем использовать функцию потерь [`sparse_categorical_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/backend/sparse_categorical_crossentropy), поскольку она использует целые числа для задач категориальной классификации. Мы хотим вычислить потери, используя истинные данные -- `labels` -- и прогнозируемые данные -- `logits`.

Давайте сначала вычислим потери, используя наш пример прогнозов для нетренированной модели:

In [ ]:
### Создадим функцию потерь ###

'''TODO: Создайте функцию, высчитывающую разницу между истинными данными
   и предсказаниями (logits). Установите аргумент from_logits=True.'''
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

  return loss

'''TODO: Вычислить потери, используя истинные данные о следующих символах 
   из тестового пакета и прогнозы нетренированной модели 
   несколькими ячейками выше'''
example_batch_loss = compute_loss(y, pred)


print("Форма предсказаний: ", pred.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Начнем с определения некоторых гиперпараметров для обучения модели.Для начала мы установим разумные значения для некоторых параметров. Вы должны использовать то, чему научились в классе, чтобы оптимизировать выбор параметров здесь!

In [ ]:
### Настройка гиперпараметров и оптимизация ###

# Оптимизация параметров:
num_training_iterations = 2000  # Увеличьте это, чтобы модель дольше обучалась
batch_size = 4  # Поэкспериментируйте со значениями от 1 до 64
seq_length = 100  # Поэкспериментируйте со значениями от 50 до 500
learning_rate = 5e-3  # Поэкспериментируйте со значениями от 1e-5 до 1e-1

# Параметры модели: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Поэкспериментируйте со значениями от 1 до 2048

# Расположение контрольных точек: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

Теперь мы готовы определить оптимизатор и длительность обучения. Поэкспериментируйте с этими параметрами, чтобы увидеть как они влияют на выход нейронной сети. Некоторые оптимизаторы, которы вы можете попробовать это [`Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam?version=stable) и [`Adagrad`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adagrad?version=stable).

Сначала мы создадим новую модель, а затем используем метод
[`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) для обратного распространения ошибки. 
Мы так же будем выводить прогресс обучения модели, что поможет нам увидить уменьшаются ли ошибки.

In [ ]:
### Определите оптимизатор и обучающую операцию ###

'''TODO: Создайте новую модель для обучения с помощью функции `build_model`
   и гиперпараметров, определенных выше.'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)


'''TODO: Создайте оптимизатор с указанием learning_rate.
  Список поддерживаемых оптимизаторов можно найти на веб-сайте tensorflow.
  https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
  Попробуйте для начала использовать оптимизатор Adam.'''
optimizer = tf.keras.optimizers.Adam(learning_rate)


@tf.function
def train_step(x, y): 
  
  with tf.GradientTape() as tape:
  
    '''TODO: Передайте вход в модель для генерации прогноза'''
    y_hat = model(x) # TODO
  
    '''TODO: Вычислите потери!'''
    loss = compute_loss(y, y_hat) # TODO

  # Теперь вычислите градиенты
  '''TODO: Помните, что мы хотим, чтобы градиент потерь учитывал 
     все параметры модели
     HINT: Используйте `model.trainable_variables` чтобы получить список 
     всех параметров модели.'''
  grads = tape.gradient(loss, model.trainable_variables) # TODO
  
  # Примените градиенты к оптимизатору, чтобы он мог 
  # соответствующим образом обновить модель
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

###################
# Начало обучения!#
###################

history = []
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # очистить, если 
#'_instances' существует в tqdm

for iter in tqdm(range(num_training_iterations)):

  # Распространить пакет по сети
  x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
  loss = train_step(x_batch, y_batch)

  # Обновить индикатор выполнения
  history.append(loss.numpy().mean())
  plotter.plot(history)

  # Обновить модель с изменёнными весами!
  if iter % 100 == 0:     
    model.save_weights(checkpoint_prefix)
    
# Сохранить обученную модель и веса
model.save_weights(checkpoint_prefix)


## 2.6 Создание музыки с помощью RNN модели

Теперь мы можем создавать музыку с помощью нашей обученной модели! Что бы начать нам нужно передать в модель какой-нибудь seed (потому что модель не сможет ничего предсказать, если не с чего будет начать).

Когда у нас будет сгенерированный seed мы сможем последовательно предсказать каждый последующий символ (помните, мы используем нотацию ABC для нашей музыки), используя обученную модель. В частности вспомним, что на выходе нашей RNN сети применяется функция `softmax` к возможным последовательным символам. Для вывода мы последовательно отбираем образцы из этих распределений, а затем используем наши образцы для кодирования сгенерированной песни в нотации ABC.

Затем все, что нам нужно сделать, это записать это в файл и прослушать!

### Восстановление последних контрольных точек.

Чтобы упростить этот шаг, мы будем использовать размер пакета равным 1. Из-за того, как внутреннее состояние в RNN передается от шага к шагу, модель сможет принять фиксированный размер пакета только после её создания. 

Чтобы запустить модель с другим batch_size, нам нужно перестроить модель и восстановить веса из последней контрольной точки, то есть веса после последней контрольной точки во время обучения:

In [ ]:
'''TODO: Перестройте модель, используя batch_size=1'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # TODO

# Восстановите веса модели из последней контрольной точки после обучения
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Обратите внимание, что мы передали фиксированный размер пакета `batch_size` равный 1.

### Процедура прогнозирования

Теперь мы готовы написать код для генерации текста в нотации ABC:

* Инициализируйте начальную строку "seed" и состояние RNN, а также установите количество символов, которые мы хотим сгенерировать. 

* Используйте начальную строку и состояние RNN, чтобы получить распределение вероятностей для следующего предсказанного символа.

* Получите выборку из полиномиального распределения для вычисления индекса предсказанного символа. Этот символ затем будет использован как следующий вход.

* На каждом шаге обновленное состояние RNN возвращается в модель, так что теперь модель имеет больше контекста для следующего прогноза. После предсказания следующего символа обновленное состояние RNN снова передается в модель, и именно так она находит закономерности последовательностей в данных, поскольку получает больше информации из предыдущих предсказаний.

![LSTM inference](https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab1/img/lstm_inference.png)

Поэкспериментируйте с некоторыми аспектами определения и обучения сети и завершите этот блок кода. И посмотрите, как работает модель. Как песни, созданные после обучения с небольшим количеством эпох отличаются от песен,  созданных после более длительного обучения?

In [ ]:
### Предсказание сгенерированной муызки ###

def generate_text(model, start_string, generation_length=1000):
  # Этап оценки (создание текста в нотации ABC, используя обученную модель)

  '''TODO: Преобразуйте начальную строку в цифры (векторизация)'''
  input_eval = [char2idx[s] for s in start_string] # TODO
  input_eval = tf.expand_dims(input_eval, 0)

  # Пустая строка для хранения наших результатов
  text_generated = []

  # Размер пакета == 1
  model.reset_states()
  tqdm._instances.clear()

  for i in tqdm(range(generation_length)):
      '''TODO: оценить входные данные и сгенерировать
         прогнозы для следующих символв'''
      predictions = model(input_eval)
      
      # Удалить размерность пакета
      predictions = tf.squeeze(predictions, 0)
      
      '''TODO: используйте полиномиальное распределение для выборки'''
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # Передайте прогноз вместе с предыдущим скрытым состоянием
      #   в качестве следующих входов в модель
      input_eval = tf.expand_dims([predicted_id], 0)
      
      '''TODO: добавить предсказанный символ к сгенерированному тексту!'''
      # Hint: подумайте, в каком формате находится прогноз по сравнению с выходом
      text_generated.append(idx2char[predicted_id]) # TODO 
    
  return (start_string + ''.join(text_generated))

In [ ]:
'''TODO: Используйте модель и функции, определенные выше, чтобы сгенерировать
   текст в нотации ABC длиной 1000!
   Как вы могли заметить файлы в нотации ABC начинаются 
   с символа "X" - это может быть хорошей начальной строкой.'''
generated_text = generate_text(model, start_string="X", generation_length=1000) # TODO

### Воспроизведите сгенерированную музыку!

Теперь мы можем вызвать функцию для преобразования текса в нотации ABC в аудио файл, и прослушать его! Попробуйте обучать модель дольше, если полученная песня окажется недостаточно длинной, или заново сгенерируйте песню!

In [ ]:
### Воспроизвести сгенерированные песни ###

generated_songs = mdl.lab1.extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs): 
  # Синтез песни
  waveform = mdl.lab1.play_song(song)

  # Если это корректная песня (правильный синтаксис), давайте ее воспроизведем!
  if waveform:
    print("Generated song", i)
    ipythondisplay.display(waveform)

## 2.7 Экспериментируйте и **получайте награды за лучшие песни** !!

Поздравляем с созданием вашей первой sequence модели с TensorFlow! Это довольно большое достижение, и, надеюсь, у вас есть какие-нибудь приятные мелодии, чтобы продемонстрировать их.

Если хотите пойти дальше, попробуйте оптимизировать свою модель и представить свою лучшую песню! Напишите нам в Твиттере [@MITDeepLearning](https://twitter.com/MITDeepLearning) или [отправьте по email](mailto:introtodeeplearning-staff@mit.edu) копию песни (если у вас нет Твиттера), и мы раздадим призы нашим фаворитам!

Подумайте, как вы можете улучшить свою модель и что кажется наиболее важным с точки зрения результата. Вот несколько идей для начала:

*  Как количество эпох влияет на результат?
*  Что, если вы измените или дополните набор данных? 
*  Сильно ли влияет выбор начальной строки на результат?

Получайте удовольствие и приятного прослушивания!


![Let's Dance!](http://33.media.tumblr.com/3d223954ad0a77f4e98a7b87136aa395/tumblr_nlct5lFVbF1qhu7oio1_500.gif)


